In [8]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import multiprocessing as mp
import glob
import sys, os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
tqdm.pandas()
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
ACTIONS = ["read_comment", "like", "click_avatar",  "forward", "comment", "follow", "favorite"]
ACTIONS2 = ACTIONS + ['finish', 'stay']
HIS_ACTIONS = ACTIONS2 + [
    'pos', 'neg', 'unfinish', 'unstay'
]

In [3]:
# FAST = True
# if FAST:
  

In [4]:
day = 14.5
emb_dim = 128
window = 32
min_count = 1

In [5]:
from gensim.models.callbacks import CallbackAny2Vec

class MonitorCallback(CallbackAny2Vec):
  def __init__(self, name):
    self.name = name
    self.epoch = 1
    self.timer = gezi.Timer()
    
  def on_epoch_end(self, model):
    # TODO 为什么打印train loss一直是0
    print('name:', self.name, 'epoch:', self.epoch, "model loss:", model.get_latest_training_loss(), f'elapsed minutes: {self.timer.elapsed_minutes():.2f}')  # print loss
    self.epoch += 1

In [6]:
vocab_names = [
                'user', 'doc',
                'author', 'singer', 'song',
                'key', 'tag', 'word', 'char'
              ]
vocabs = {}

for vocab_name in vocab_names:
  vocab_file =  f'../input/{vocab_name}_vocab.txt'
  vocab = gezi.Vocab(vocab_file)
  vocabs[vocab_name] = vocab


In [7]:
def gen_w2v(name, day, window=32, min_count=1):
  sentences = []
  if day:
    file = f'../input/{day}/{name}_corpus.txt'
  else:
    file = f'../input/{name}_corpus.txt'
  for line in open(file):
    l = line.rstrip().split()
    sentences.append(l)
  monitor = MonitorCallback(name) 
  w2v = Word2Vec(sentences, vector_size=emb_dim, window=window, min_count=min_count, sg=1, workers=mp.cpu_count(), epochs=10, callbacks=[monitor])
  vocab = vocabs[name]
  emb = np.zeros([vocab.size(), emb_dim])
  # emb = np.random.uniform(-0.05, 0.05,(vocab.size(), emb_dim))
  for i in range(vocab.size()):
    word = vocab.key(i) 
    if word in w2v.wv:
      emb[i] = w2v.wv[word]
  if day:
    ofile = f'../input/{day}/{name}_w2v{window}_{min_count}_emb.npy'
  else:
    ofile = f'../input/{name}_w2v{window}_{min_count}_emb.npy'
  np.save(ofile, emb)
  # print(emb.shape)

In [8]:
names = [
  'doc', 
  'user',
  'author', 
  'singer', 
  'song',
  ]
# days = [14.5, 15, 13, 14]
days = [14.5, 13]
if 'tione' in os.environ['PATH']:
  days = [14.5, 15]
for day in tqdm(days):
  t = tqdm(names, desc=f'{day}')
  for name in t:
    t.set_postfix({'name': name, 'day': day, 'window': window, 'min_count': min_count})
    gen_w2v(name, day, window=window, min_count=min_count)
    # gen_w2v(name, day, window=32, min_count=1) # 0.6828
    # gen_w2v(name, day, window=32, min_count=5) # 0.6825
    # gen_w2v(name, day, window=64, min_count=1) # 0.6829
    # gen_w2v(name, day, window=128, min_count=1)  # 0.6852
    # gen_w2v(name, day, window=256, min_count=1)

  0%|          | 0/4 [00:00<?, ?it/s]

14.5:   0%|          | 0/5 [00:00<?, ?it/s]

name: doc epoch: 1 model loss: 0.0 elapsed minutes: 0.24
name: doc epoch: 2 model loss: 0.0 elapsed minutes: 0.20
name: doc epoch: 3 model loss: 0.0 elapsed minutes: 0.16
name: doc epoch: 4 model loss: 0.0 elapsed minutes: 0.12
name: doc epoch: 5 model loss: 0.0 elapsed minutes: 0.13
name: doc epoch: 6 model loss: 0.0 elapsed minutes: 0.13
name: doc epoch: 7 model loss: 0.0 elapsed minutes: 0.13
name: doc epoch: 8 model loss: 0.0 elapsed minutes: 0.12
name: doc epoch: 9 model loss: 0.0 elapsed minutes: 0.13
name: doc epoch: 10 model loss: 0.0 elapsed minutes: 0.12
name: user epoch: 1 model loss: 0.0 elapsed minutes: 0.15
name: user epoch: 2 model loss: 0.0 elapsed minutes: 0.11
name: user epoch: 3 model loss: 0.0 elapsed minutes: 0.12
name: user epoch: 4 model loss: 0.0 elapsed minutes: 0.11
name: user epoch: 5 model loss: 0.0 elapsed minutes: 0.12
name: user epoch: 6 model loss: 0.0 elapsed minutes: 0.12
name: user epoch: 7 model loss: 0.0 elapsed minutes: 0.12
name: user epoch: 8 mod

15:   0%|          | 0/5 [00:00<?, ?it/s]

name: doc epoch: 1 model loss: 0.0 elapsed minutes: 0.26
name: doc epoch: 2 model loss: 0.0 elapsed minutes: 0.19
name: doc epoch: 3 model loss: 0.0 elapsed minutes: 0.19
name: doc epoch: 4 model loss: 0.0 elapsed minutes: 0.19
name: doc epoch: 5 model loss: 0.0 elapsed minutes: 0.20
name: doc epoch: 6 model loss: 0.0 elapsed minutes: 0.22
name: doc epoch: 7 model loss: 0.0 elapsed minutes: 0.22
name: doc epoch: 8 model loss: 0.0 elapsed minutes: 0.21
name: doc epoch: 9 model loss: 0.0 elapsed minutes: 0.22
name: doc epoch: 10 model loss: 0.0 elapsed minutes: 0.20
name: user epoch: 1 model loss: 0.0 elapsed minutes: 0.16
name: user epoch: 2 model loss: 0.0 elapsed minutes: 0.12
name: user epoch: 3 model loss: 0.0 elapsed minutes: 0.14
name: user epoch: 4 model loss: 0.0 elapsed minutes: 0.12
name: user epoch: 5 model loss: 0.0 elapsed minutes: 0.11
name: user epoch: 6 model loss: 0.0 elapsed minutes: 0.11
name: user epoch: 7 model loss: 0.0 elapsed minutes: 0.11
name: user epoch: 8 mod

13:   0%|          | 0/5 [00:00<?, ?it/s]

name: doc epoch: 1 model loss: 0.0 elapsed minutes: 0.20
name: doc epoch: 2 model loss: 0.0 elapsed minutes: 0.15
name: doc epoch: 3 model loss: 0.0 elapsed minutes: 0.15
name: doc epoch: 4 model loss: 0.0 elapsed minutes: 0.14
name: doc epoch: 5 model loss: 0.0 elapsed minutes: 0.17
name: doc epoch: 6 model loss: 0.0 elapsed minutes: 0.16
name: doc epoch: 7 model loss: 0.0 elapsed minutes: 0.15
name: doc epoch: 8 model loss: 0.0 elapsed minutes: 0.14
name: doc epoch: 9 model loss: 0.0 elapsed minutes: 0.17
name: doc epoch: 10 model loss: 0.0 elapsed minutes: 0.15
name: user epoch: 1 model loss: 0.0 elapsed minutes: 0.10
name: user epoch: 2 model loss: 0.0 elapsed minutes: 0.09
name: user epoch: 3 model loss: 0.0 elapsed minutes: 0.08
name: user epoch: 4 model loss: 0.0 elapsed minutes: 0.09
name: user epoch: 5 model loss: 0.0 elapsed minutes: 0.08
name: user epoch: 6 model loss: 0.0 elapsed minutes: 0.08
name: user epoch: 7 model loss: 0.0 elapsed minutes: 0.08
name: user epoch: 8 mod

14:   0%|          | 0/5 [00:00<?, ?it/s]

name: doc epoch: 1 model loss: 0.0 elapsed minutes: 0.23
name: doc epoch: 2 model loss: 0.0 elapsed minutes: 0.18
name: doc epoch: 3 model loss: 0.0 elapsed minutes: 0.16
name: doc epoch: 4 model loss: 0.0 elapsed minutes: 0.18
name: doc epoch: 5 model loss: 0.0 elapsed minutes: 0.18
name: doc epoch: 6 model loss: 0.0 elapsed minutes: 0.16
name: doc epoch: 7 model loss: 0.0 elapsed minutes: 0.18
name: doc epoch: 8 model loss: 0.0 elapsed minutes: 0.16
name: doc epoch: 9 model loss: 0.0 elapsed minutes: 0.17
name: doc epoch: 10 model loss: 0.0 elapsed minutes: 0.18
name: user epoch: 1 model loss: 0.0 elapsed minutes: 0.13
name: user epoch: 2 model loss: 0.0 elapsed minutes: 0.10
name: user epoch: 3 model loss: 0.0 elapsed minutes: 0.10
name: user epoch: 4 model loss: 0.0 elapsed minutes: 0.10
name: user epoch: 5 model loss: 0.0 elapsed minutes: 0.10
name: user epoch: 6 model loss: 0.0 elapsed minutes: 0.10
name: user epoch: 7 model loss: 0.0 elapsed minutes: 0.10
name: user epoch: 8 mod

In [16]:
names = [
  'word', 
  'tag',
  'key',
  'char'
]
t = tqdm(names, desc=f'{day}')
for name in t:
  t.set_postfix({'name': name})
  gen_w2v(name, 0, window=8, min_count=1)

14.5:   0%|          | 0/1 [00:00<?, ?it/s]

name: char epoch: 1 model loss: 0.0 elapsed minutes: 3.22
name: char epoch: 2 model loss: 0.0 elapsed minutes: 2.67
name: char epoch: 3 model loss: 0.0 elapsed minutes: 2.60
name: char epoch: 4 model loss: 0.0 elapsed minutes: 2.95
name: char epoch: 5 model loss: 0.0 elapsed minutes: 3.08
name: char epoch: 6 model loss: 0.0 elapsed minutes: 3.46
name: char epoch: 7 model loss: 0.0 elapsed minutes: 3.06
name: char epoch: 8 model loss: 0.0 elapsed minutes: 3.36
name: char epoch: 9 model loss: 0.0 elapsed minutes: 3.24
name: char epoch: 10 model loss: 0.0 elapsed minutes: 3.25


In [12]:
# d = pd.read_csv('../input/feed_info.csv')
# NAN_ID = -1
# d = d.fillna(NAN_ID)

In [26]:
# desc_documents = []
# count = 0
# for i, desc in tqdm(enumerate(d.description.values), total=len(d)):
#   if str(desc) == str(NAN_ID):
#     continue
#   desc_documents.append(TaggedDocument(desc.split(), [count]))
#   count += 1
# monitor = MonitorCallback('desc') 
# model = Doc2Vec(desc_documents, vector_size=emb_dim, window=8, min_count=1, workers=mp.cpu_count(), callbacks=[monitor])
# model.save('../input/desc_doc2vec.model')

  0%|          | 0/106444 [00:00<?, ?it/s]

name: char epoch: 1 model loss: 0.0 elapsed minutes: 0.31
name: char epoch: 2 model loss: 0.0 elapsed minutes: 0.26
name: char epoch: 3 model loss: 0.0 elapsed minutes: 0.22
name: char epoch: 4 model loss: 0.0 elapsed minutes: 0.26
name: char epoch: 5 model loss: 0.0 elapsed minutes: 0.26
name: char epoch: 6 model loss: 0.0 elapsed minutes: 0.25
name: char epoch: 7 model loss: 0.0 elapsed minutes: 0.24
name: char epoch: 8 model loss: 0.0 elapsed minutes: 0.24
name: char epoch: 9 model loss: 0.0 elapsed minutes: 0.26
name: char epoch: 10 model loss: 0.0 elapsed minutes: 0.26


In [27]:
# ocr_documents = []
# count = 0
# for i, ocr in tqdm(enumerate(d.ocr.values), total=len(d)):
#   if str(ocr) == str(NAN_ID):
#     continue
#   desc_documents.append(TaggedDocument(ocr.split(), [count]))
#   count += 1
# monitor = MonitorCallback('ocr') 
# model = Doc2Vec(desc_documents, vector_size=emb_dim, window=8, min_count=1, workers=mp.cpu_count(), callbacks=[monitor])
# model.save('../input/ocr_doc2vec.model')

  0%|          | 0/106444 [00:00<?, ?it/s]

name: ocr epoch: 1 model loss: 0.0 elapsed minutes: 0.57
name: ocr epoch: 2 model loss: 0.0 elapsed minutes: 0.47
name: ocr epoch: 3 model loss: 0.0 elapsed minutes: 0.44
name: ocr epoch: 4 model loss: 0.0 elapsed minutes: 0.45
name: ocr epoch: 5 model loss: 0.0 elapsed minutes: 0.43
name: ocr epoch: 6 model loss: 0.0 elapsed minutes: 0.46
name: ocr epoch: 7 model loss: 0.0 elapsed minutes: 0.45
name: ocr epoch: 8 model loss: 0.0 elapsed minutes: 0.56
name: ocr epoch: 9 model loss: 0.0 elapsed minutes: 0.44
name: ocr epoch: 10 model loss: 0.0 elapsed minutes: 0.44


In [28]:
# asr_documents = []
# count = 0
# for i, asr in tqdm(enumerate(d.asr.values), total=len(d)):
#   if str(asr) == str(NAN_ID):
#     continue
#   desc_documents.append(TaggedDocument(asr.split(), [count]))
#   count += 1
# monitor = MonitorCallback('asr') 
# model = Doc2Vec(desc_documents, vector_size=emb_dim, window=8, min_count=1, workers=mp.cpu_count(), callbacks=[monitor])
# model.save('../input/asr_doc2vec.model')

  0%|          | 0/106444 [00:00<?, ?it/s]

name: asr epoch: 1 model loss: 0.0 elapsed minutes: 0.78
name: asr epoch: 2 model loss: 0.0 elapsed minutes: 0.67
name: asr epoch: 3 model loss: 0.0 elapsed minutes: 0.68
name: asr epoch: 4 model loss: 0.0 elapsed minutes: 0.67
name: asr epoch: 5 model loss: 0.0 elapsed minutes: 0.67
name: asr epoch: 6 model loss: 0.0 elapsed minutes: 0.65
name: asr epoch: 7 model loss: 0.0 elapsed minutes: 0.65
name: asr epoch: 8 model loss: 0.0 elapsed minutes: 0.66
name: asr epoch: 9 model loss: 0.0 elapsed minutes: 0.65
name: asr epoch: 10 model loss: 0.0 elapsed minutes: 0.66


In [9]:
# vocab = vocabs['doc']

In [16]:
# model = Doc2Vec.load('../input/desc_doc2vec.model')
# emb = np.zeros([vocab.size(), emb_dim])
# for i, desc in tqdm(enumerate(d.description.values), total=len(d)):
#   if str(desc) == str(NAN_ID):
#     continue
#   emb[vocab.id(int(d.feedid.values[i]))] = model.infer_vector(desc.split())
# np.save('../input/desc_vec_emb.npy', emb)

  0%|          | 0/106444 [00:00<?, ?it/s]

In [17]:
# model = Doc2Vec.load('../input/ocr_doc2vec.model')
# emb = np.zeros([vocab.size(), emb_dim])
# for i, ocr in tqdm(enumerate(d.ocr.values), total=len(d)):
#   if str(ocr) == str(NAN_ID):
#     continue
#   emb[i] = model.infer_vector(ocr.split())
# np.save('../input/ocr_vec_emb.npy', emb)

  0%|          | 0/106444 [00:00<?, ?it/s]

In [18]:
# model = Doc2Vec.load('../input/asr_doc2vec.model')
# emb = np.zeros([vocab.size(), emb_dim])
# for i, asr in tqdm(enumerate(d.asr.values), total=len(d)):
#   if str(asr) == str(NAN_ID):
#     continue
#   emb[i] = model.infer_vector(asr.split())
# np.save('../input/asr_vec_emb.npy', emb)

  0%|          | 0/106444 [00:00<?, ?it/s]